In [1]:
import sys;sys.path.append('d:/code/pywfn')

# 文件读取
文件读取的核心任务是从化学文件中提取出计算分子性质时需要的信息，主要分为三类

- 结构信息
- 基组信息
- 轨道信息

所有的读取器都包含在`pywfn.reader`中，当前支持的读取器及支持的信息如下：

|读取类|结构信息|基组信息|轨道信息|说明|
|:---:|:---:|:---:|:---:|:---:|
|`LogReader`|√|?|?|读取高斯的输出文件|
|`GjfReader`|√|×|×|读取高斯的输入文件|
|`FchReader`|√|√|√|读取高斯的检查点文件|
|`ModReader`|√|√|√|读取`.molden`文件|
|`XyzReader`|√|×|×||
|`MolReader`|√|×|×||
|`SdfReader`|√|×|×||
|`AnyReader`|×|×|×|可自定义信息|
## 信息详情

### 结构信息

包含原子的`类型`和`坐标`，以常见的高斯输入文件类型`gjf`为例

```
%chk=D:\gfile\C6H6.chk
# b3lyp/6-31g(d) pop=full gfinput iop(3/33=1)

Title Card Required

0 1
 C                  0.00000000    1.40140000   -0.00000000
 C                 -1.21364800    0.70070000   -0.00000000
 C                 -1.21364800   -0.70070000   -0.00000000
 C                  0.00000000   -1.40140000   -0.00000000
 C                  1.21364800   -0.70070000   -0.00000000
 C                  1.21364800    0.70070000   -0.00000000
 H                  0.00000000    2.47140000   -0.00000000
 H                 -2.14029518    1.23570000   -0.00000000
 H                 -2.14029518   -1.23570000   -0.00000000
 H                  0.00000000   -2.47140000   -0.00000000
 H                  2.14029518   -1.23570000   -0.00000000
 H                  2.14029518    1.23570000   -0.00000000


```
其中存储着原子的元素符号，即为原子类型，以及原子的xyz坐标，即原子坐标。

不同类型的文件存储使用的原子坐标的单位不同，可以为`Angstorm`或`Borh`，pywfn内部统一转为`Borh`单位。

### 基组信息
包含高斯型基函数的系数和指数

以`6-31g(d)`基组为例：

![基组示例](./imgs/基组示例.png)

- `654`行中的`1`代表当前基组信息属于第一个原子
- `655`行中`S`代表`1S`轨道，`6`代表`1S`轨道有`6`个GTO函数线性组合而成
- `656-611`中两列数据分别代表GTO函数的`指数`和`系数`
- `662`行中的`SP`代表`2S`和`2P`轨道，分别也是有`6`个GTO函数线性组合而成
- `663-668`中，第一列和第二列为`2S`和`2P`的GTO指数，第三列代表两者共享的系数
- 其它的也都是类似的

### 轨道信息


轨道信息主要就是与分子轨道相关的信息，主要包含：

- 轨道系数矩阵。分为开窍层和闭壳层。形状为[nato,nobt]，对于闭壳层，nato=nobt；对于开窍层，nato*2=nobt
- 分子轨道-能量。长度为nobt的浮点数列表。
- 分子轨道-占据。长度为nobt的布尔值列表。表示分子轨道是否占据
- 原子轨道-原子索引。每个原子轨道对应的原子索引
- 原子轨道-轨道壳层。原子轨道对应的壳层索引。
- 原子轨道-轨道符号。每个原子轨道的轨道符号。可以为：`S`、`PX`、`PY`、`PZ` 等等

这些信息都存储在`base.Coef`类型中

直接读取到的原子轨道信息混合了`笛卡尔型基函数`和`球谐型基函数`，在Coef类型中可以将转为全部为笛卡尔型或者全部为球谐型。

## fch文件

**示例代码**

In [5]:
from pywfn.base import Mole
from pywfn.reader import FchReader

reader=FchReader('./mols/C6H6.fch')
mol=Mole(reader)

print(mol.geome)
print(mol.basis)
print(mol.coefs)

  C   -0.0000    2.6483    0.0000
  C    2.2935    1.3241    0.0000
  C    2.2935   -1.3241    0.0000
  C    0.0000   -2.6483    0.0000
  C   -2.2935   -1.3241    0.0000
  C   -2.2935    1.3241    0.0000
  H   -0.0000    4.6703    0.0000
  H    4.0446    2.3351    0.0000
  H    4.0446   -2.3351    0.0000
  H    0.0000   -4.6703    0.0000
  H   -4.0446   -2.3351    0.0000
  H   -4.0446    2.3351    0.0000
  idx  atm  shl  ang         coe         alp
    1    1    1    0  1.8347e-03  3.0475e+03
    2    1    1    0  1.4037e-02  4.5737e+02
    3    1    1    0  6.8843e-02  1.0395e+02
    4    1    1    0  2.3218e-01  2.9210e+01
    5    1    1    0  4.6794e-01  9.2867e+00
    6    1    1    0  3.6231e-01  3.1639e+00
    7    1    2    0 -1.1933e-01  7.8683e+00
    8    1    2    0 -1.6085e-01  1.8813e+00
    9    1    2    0  1.1435e+00  5.4425e-01
   10    1    2    1  6.8999e-02  7.8683e+00
   11    1    2    1  3.1642e-01  1.8813e+00
   12    1    2    1  7.4431e-01  5.4425e-01
   13  

## gjf文件

**示例代码**

In [4]:
from pywfn.base import Mole
from pywfn.reader import GjfReader

reader=GjfReader('./mols/C6H6.gjf')
mol=Mole(reader)

print(mol.geome)

  C    0.0000    2.6472    0.0000
  C    2.2926    1.3236    0.0000
  C    2.2926   -1.3236    0.0000
  C    0.0000   -2.6472    0.0000
  C   -2.2926   -1.3236    0.0000
  C   -2.2926    1.3236    0.0000
  H    0.0000    4.6685    0.0000
  H    4.0430    2.3342    0.0000
  H    4.0430   -2.3342    0.0000
  H    0.0000   -4.6685    0.0000
  H   -4.0430   -2.3342    0.0000
  H   -4.0430    2.3342    0.0000
